<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/MLBDataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!wget -q https://github.com/profteachkids/StemUnleashed/raw/main/mlb_batting.zip -O mlb_batting.zip

In [14]:
import numpy as np
import pandas as pd
import shutil
from pathlib import Path

In [15]:
shutil.unpack_archive('mlb_batting.zip',extract_dir='mlb_batting')

In [16]:
files=list(Path('mlb_batting').glob('**/*.csv'))
keys=[afile.as_posix().split('/')[1:] for afile in files]
keys=[tuple((*key[:-1],key[-1].split('.')[0])) for key in keys]

In [26]:
df_list=[]
days=['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
for afile in files:
    df= pd.read_csv(afile).iloc[:,1:]
    df['home']=df['OPP'].str.startswith('vs').astype(int)
    df['win']=df['Result'].str.startswith('W').astype(int)
    df=df.loc[df['Date'].str[:3].isin(days),['Date','home','AB','HR', 'win']]
    df[['day','Date']]=df.loc[:,'Date'].str.split(' ',expand=True).loc[:,:1]
    df_list.append(df)

In [27]:
df=pd.concat(df_list,keys=keys)
df.index=df.index.droplevel(3)
df.index.names=['team','player','year']
df=df.reset_index().set_index(['team','player'])
df[['AB','HR']]=df[['AB','HR']].astype(int)
df['day']=df['day'].astype('category')

In [28]:
#combine month/day with year
df['date']=pd.to_datetime(df['Date']+'/' +df['year'])
df=df.drop(['Date','year'],axis=1)

In [29]:
df_team=df.pivot_table(values=['AB','HR'],index=['team','home'],aggfunc=['sum'])

In [30]:
df_team.columns=df_team.columns.droplevel(0)

In [31]:
df_team['HR_percentage']=df_team['HR']/df_team['AB']

In [32]:
df_ratio=pd.pivot_table(df_team, index='team',columns='home',values='HR_percentage')

In [35]:
df_ratio['home_away_ratio']=df_ratio.loc[:,1]/df_ratio.loc[:,0]

In [37]:
df_ratio.sort_values('home_away_ratio',ascending=False)

home,0,1,home_away_ratio
team,,,
colorado-rockies,0.033055,0.042727,1.292609
chicago-white-sox,0.032907,0.039290,1.193999
baltimore-orioles,0.035093,0.041690,1.187971
san-diego-padres,0.035341,0.041202,1.165843
los-angeles-dodgers,0.038326,0.044121,1.151208
seattle-mariners,0.039068,0.044925,1.149908
boston-red-sox,0.035879,0.040978,1.142131
houston-astros,0.032262,0.036200,1.122068
cincinnati-reds,0.035382,0.039470,1.115548
